# D. Explore local attributes, local parameters (global hru) and basin parameters (global gru) of SUMMA 3.0

## 1. Import pySUMMA 3.0.0

In [1]:
import pysumma as ps

## 2. Create Simulatioin Object

In [2]:
executable = "/usr/bin/summa.exe"
file_manager = 'SummaModel_ReynoldsAspenStand_StomatalResistance/settings/summa_fileManager_riparianAspenSimpleResistance.txt'

s = ps.Simulation(executable, file_manager)

## 3. Attribute and Parameter files

SUMMA uses a number of files to specify model attributes and parameters. Although SUMMA's distinction between attributes and parameters is somewhat arbitrary, attributes generally describe characteristics of the model domain that are time-invariant during the simulation, such as GRU and HRU identifiers, spatial organization, an topography. The important part for understanding the organization of the SUMMA input files is that the values specified in the **`local attributes`** file do not overlap with those in the various parameter files. Thus, these values do not overwrite any attributes specified elsewhere. In contrast, the various parameter file are read in sequence (as explained in the next paragraph) and parameter values that are read in from the input files successively overwrite values that have been specified earlier.

The figure below shows the order in which SUMMA processes the various attribute and parameter files. First, the **`local attributes file`** is processed, which provides information about the organization of the GRUs and HRUs as well as some other information. Then, SUMMA parses the **`local parameters file`**, which provides spatially constant values for all SUMMA parameters that need to be specified at the HRU level. SUMMA then parses the **`basin parameters file`**, which provides spatially constant values for all SUMMA parameters that need to be specified at the GRU level. In this case, it does not really matter which files is parsed first. The information in these two files does not overlap. At this point in SUMMA's initialization, all GRU and HRU parameters have been initialized to spatially constant values. SUMMA has inherited some routines from the NOAH land surface model and the next step is to parse the **`NOAH parameter tables`**, based on the spatial information specified in the **`local attributes file`**. The information in these tables is used to overwrite the spatially constant values that have already been initialized for each HRU. Finally, the **`trial parameters file`** is parsed to provide additional GRU and HRU specific information. The values from this file will overwrite existing values. The number of variables specified in the trial parameters file will vary with the amount of location-specific information that you have available for your simulation.

<img src="order.PNG" width="800">

## 4. Local attributes file

The local attributes file is a NetCDF file that specifies model element attributes for GRUs and individual HRUs. The local attributes file is parsed by **`build/source/driver/multi_driver.f90`** and **`build/source/engine/read_attrb.f90`**. As described above, the attributes specified in this file are separate from the values specified in the various parameter files.

The local attributes file contains a **`gru`** and an **`hru`** dimension as specified in the table below. All variables in the local attributes file must be specified.

|Variable|	dimension|	type|	units|	long name|	notes|
|:----------|:---------------|:----------|:----------|:-------------|:-------------|
|hruId|	hru|	int|	-	|Index of hydrological response unit (HRU)	|Unique numeric ID for each HRU|
|gruId|	gru|	int|	-	|Index of grouped response unit (GRU)|	Unique numeric ID for each GRU|
|hru2gruId|	hru|	int|	-	|Index of GRU to which the HRU belongs|	gruId of the GRU to which the HRU belongs|
|downHRUindex|	hru|	int|	-	|Index of downslope HRU (0 = basin outlet)	|Downslope HRU must be within the same GRU. If the value is 0, then there is no exchange to a neighboring HRU. Setting this value to 0 for all HRUs emulates a series of independent columns|
|longitude|	hru|	double|	Decimal degree east|	Longitude of HRUs centroid	|West is negative or greater than 180|
|latitude|	hru|	double|	Decimal degree north|	Latitude of HRUs centroid|	South is negative|
|elevation|	hru|	double|	m	|Elevation of HRUs centroid	||
|HRUarea|	hru|	double|	m^2|	Area of HRU	||
|tan_slope|	hru|	double|	m m-1|	Average tangent slope of HRU	||
|contourLength|	hru|	double|	m|	Contour length of HRU	|Width of a hillslope (m) parallel to a stream. Used in groundwatr.f90|
|slopeTypeIndex|	hru|	int|	-	|Index defining slope	||
|soilTypeIndex|	hru|	int|	-	|Index defining soil type	||
|vegTypeIndex|	hru|	int|	-	|Index defining vegetation type	||
|mHeight|	hru|	double|	m	|Measurement height above bare ground	||

Below is a sample layout of the local attributes file (the output of running `ncdump -h`). In this case, both the gru and hru dimension are of size 1 (the example is taken from one of the test cases, most of which are point model simulations), but of course there can be many GRUs and HRUs.

### 4.1 Get name of Local Attribute file

In [3]:
s.manager['attributeFile'].value

'summa_zLocalAttributes_riparianAspen.nc'

### 4.2 Show Data of Local Attribute file

In [4]:
# Read local attributes netCDF file
print(s.local_attributes)

<xarray.Dataset>
Dimensions:         (gru: 1, hru: 1)
Dimensions without coordinates: gru, hru
Data variables:
    hru2gruId       (hru) int32 1001
    downHRUindex    (hru) int32 0
    slopeTypeIndex  (hru) int32 1
    soilTypeIndex   (hru) int32 8
    vegTypeIndex    (hru) int32 11
    mHeight         (hru) float64 19.25
    contourLength   (hru) float64 180.8
    tan_slope       (hru) float64 0.2083
    elevation       (hru) float64 2.097e+03
    longitude       (hru) float64 243.2
    latitude        (hru) float64 43.2
    HRUarea         (hru) float64 3.27e+04
    hruId           (hru) int32 1001
    gruId           (gru) int32 1001
Attributes:
    datasource:  *zLocalAttributes*
    history:     Tue Sep 27 21:51:50 2016: ncrename -d nhru,hru -d ngru,gru ....
    NCO:         "4.6.0"


### 4.3 Check shape and change the value of Local Attribute file

In [5]:
s.local_attributes['hruId'].shape

(1,)

In [6]:
s.assign_attributes('hruId', int(1005))

In [7]:
# Read local attributes netCDF file
print(s.local_attributes)

<xarray.Dataset>
Dimensions:         (gru: 1, hru: 1)
Dimensions without coordinates: gru, hru
Data variables:
    hru2gruId       (hru) int32 1001
    downHRUindex    (hru) int32 0
    slopeTypeIndex  (hru) int32 1
    soilTypeIndex   (hru) int32 8
    vegTypeIndex    (hru) int32 11
    mHeight         (hru) float64 19.25
    contourLength   (hru) float64 180.8
    tan_slope       (hru) float64 0.2083
    elevation       (hru) float64 2.097e+03
    longitude       (hru) float64 243.2
    latitude        (hru) float64 43.2
    HRUarea         (hru) float64 3.27e+04
    hruId           (hru) int64 1005
    gruId           (gru) int32 1001
Attributes:
    datasource:  *zLocalAttributes*
    history:     Tue Sep 27 21:51:50 2016: ncrename -d nhru,hru -d ngru,gru ....
    NCO:         "4.6.0"


## Need to check the value is changed or not

In [8]:
import numpy as np
s.local_attributes['hruId'].values = np.array(int(1005)).reshape(s.local_attributes['hruId'].shape)

In [9]:
# Read local attributes netCDF file
print(s.local_attributes)

<xarray.Dataset>
Dimensions:         (gru: 1, hru: 1)
Dimensions without coordinates: gru, hru
Data variables:
    hru2gruId       (hru) int32 1001
    downHRUindex    (hru) int32 0
    slopeTypeIndex  (hru) int32 1
    soilTypeIndex   (hru) int32 8
    vegTypeIndex    (hru) int32 11
    mHeight         (hru) float64 19.25
    contourLength   (hru) float64 180.8
    tan_slope       (hru) float64 0.2083
    elevation       (hru) float64 2.097e+03
    longitude       (hru) float64 243.2
    latitude        (hru) float64 43.2
    HRUarea         (hru) float64 3.27e+04
    hruId           (hru) int64 1005
    gruId           (gru) int32 1001
Attributes:
    datasource:  *zLocalAttributes*
    history:     Tue Sep 27 21:51:50 2016: ncrename -d nhru,hru -d ngru,gru ....
    NCO:         "4.6.0"


In [10]:
s._write_configuration()

## 5. Explore Local parameters (Global HRU) file text file

The local parameters file is an ASCII file that specifies spatially constant parameter values for SUMMA parameters. The file is parsed by **`build/source/engine/read_pinit.f90:read_pinit()`**.

The first non-comment line is a format string that is used to process the remaining non-comment lines. The format definition defines the format of the file, which can be changed. The format string itself is a Fortran format statement and must be in single quotes. For example,

`'(a25,1x,3(a1,1x,f12.4,1x))' ! format string (must be in single quotes)`

This states that the first field is 25 ASCII characters (`a25`), followed by a space (`1x`), and then 3 fields that each consist of an ASCII character (`a1`, the separator), followed by a space (`1x`), a float with 4 digits after the decimal (`f12.4`, note that you can also use this to read something like `1.0d+6`), followed by another space (`1x`). For the separator we often use |, but other characters can be used as well. For example, this format statement can be used to read a line such as

```
| upperBoundHead          | -0.7500 | -100.0000 | -0.0100 |
```

All lines in the file (including the format statement) consist of four columns

1. parameter name
2. default parameter value
3. lower parameter limit
4. upper parameter limit

The parameters that need to be specified in this file are those listed as `iLook_param` in the `var_lookup` module in `build/source/dshare/var_lookup.f90` (look for the comment `(5) define model parameters`). Parameter names must match the code exactly (case-sensitive). The parameter value is set to the default value (second column). The parameter value limits are currently not used, but still need to be specified. Our intention is to use them when reading the trial parameters file to ensure that the `hru` specific parameter values are within the specified limits.

### 5.1 Show local param info using `print` command

In [12]:
print(s.global_hru_params)

upperBoundHead            |      -7.5d-1 |      -1.0d+2 |      -1.0d-2
lowerBoundHead            |       0.0000 |      -1.0d+2 |      -1.0d-2
upperBoundTheta           |       0.2004 |       0.1020 |       0.3680
lowerBoundTheta           |       0.1100 |       0.1020 |       0.3680
upperBoundTemp            |     272.1600 |     270.1600 |     280.1600
lowerBoundTemp            |     274.1600 |     270.1600 |     280.1600
tempCritRain              |     273.1600 |     272.1600 |     274.1600
tempRangeTimestep         |       2.0000 |       0.5000 |       5.0000
frozenPrecipMultip        |       1.0000 |       0.5000 |       1.5000
snowfrz_scale             |      50.0000 |      10.0000 |    1000.0000
fixedThermalCond_snow     |       0.3500 |       0.1000 |       1.0000
albedoMax                 |       0.8400 |       0.7000 |       0.9500
albedoMinWinter           |       0.5500 |       0.6000 |       1.0000
albedoMinSpring           |       0.5500 |       0.3000 |       1.0000
albedo

### 5.2 Show value of  `albedoMax` in local param info text file

In [13]:
print(s.global_hru_params['albedoMax'])

albedoMax                 |       0.8400 |       0.7000 |       0.9500


### 5.3 Change value of  `albedoMax` in local param info text file

In [14]:
s.global_hru_params['albedoMax'] = 0.7
print(s.global_hru_params['albedoMax'])

albedoMax                 |       0.7000 |       0.7000 |       0.7000


In [15]:
print(s.global_hru_params)

upperBoundHead            |      -7.5d-1 |      -1.0d+2 |      -1.0d-2
lowerBoundHead            |       0.0000 |      -1.0d+2 |      -1.0d-2
upperBoundTheta           |       0.2004 |       0.1020 |       0.3680
lowerBoundTheta           |       0.1100 |       0.1020 |       0.3680
upperBoundTemp            |     272.1600 |     270.1600 |     280.1600
lowerBoundTemp            |     274.1600 |     270.1600 |     280.1600
tempCritRain              |     273.1600 |     272.1600 |     274.1600
tempRangeTimestep         |       2.0000 |       0.5000 |       5.0000
frozenPrecipMultip        |       1.0000 |       0.5000 |       1.5000
snowfrz_scale             |      50.0000 |      10.0000 |    1000.0000
fixedThermalCond_snow     |       0.3500 |       0.1000 |       1.0000
albedoMax                 |       0.7000 |       0.7000 |       0.7000
albedoMinWinter           |       0.5500 |       0.6000 |       1.0000
albedoMinSpring           |       0.5500 |       0.3000 |       1.0000
albedo

## 6. Explore Basin parameters (Global GRU) text file

The basin parameters file is an ASCII file that specifies spatially constant parameter values for SUMMA basin parameters. The file is parsed by **`build/source/engine/read_pinit.f90:read_pinit()`**. The format of the file is identical to that of the local parameters file.

The parameters that need to be specified in this file are those listed as **`iLook_bpar`** in the **`var_lookup`** module in **`build/source/dshare/var_lookup.f90`** (look for the comment **`(11) define basin-average model parameters`**). Parameter names must match the code exactly (case-sensitive). The parameter value is set to the default value (second column). The parameter value limits are currently not used, but still need to be specified. Our intention is to use them when reading the trial parameters file to ensure that the **`gru`** specific parameter values are within the specified limits.

### 6.1 Show basin param info using `print` command

In [16]:
print(s.global_gru_params)

basin__aquiferHydCond     |       0.0100 |       0.0001 |      10.0000
basin__aquiferScaleFactor |       3.5000 |       0.1000 |     100.0000
basin__aquiferBaseflowExp |       5.0000 |       1.0000 |      10.0000
routingGammaShape         |       2.5000 |       2.0000 |       3.0000
routingGammaScale         |       2.0d+4 |       1.0000 |       5.0d+6


### 6.2 Show value of  `basin__aquiferScaleFactor` in basin param info text file

In [17]:
print(s.global_gru_params['basin__aquiferScaleFactor'])

basin__aquiferScaleFactor |       3.5000 |       0.1000 |     100.0000


### 6.3 Change value of  `basin__aquiferScaleFactor` in basin param info text file

In [18]:
s.global_gru_params['basin__aquiferScaleFactor'] = 5.0
print(s.global_gru_params['basin__aquiferScaleFactor'])

basin__aquiferScaleFactor |       5.0000 |       5.0000 |       5.0000
